#**iii. Frecuencia relativista**#
Estudio de los efectos que produce un campo magnético sobre el movimiento de una partícula con una velocidad cercana a la velocidad de la luz (c).

La ecuación de movimiento de una partícula bajo efectos relativistas es:

$$m \gamma \frac{d \vec{v}}{dt}=q \vec{v} \times  \vec{B}$$

donde $$\gamma = \frac{1}{\sqrt{1-v^2/c^2}} $$

Movimiento de particulas en el seno de un campo magnético constante. Nos vamos a centrar en el periodo del movimiento de la partícula y como varía al aplicar un factor relativista. 
Usaremos las mismas ecuaciones de movimiento circular y de la fuerza magnética,  con la única diferencia del uso de la masa relativista de la partícula:

Y para calcular el periodo:

$$T=\frac{2\pi}{w}=\frac{2\pi m \gamma}{qB}$$

Por tanto, tenemos una partícula de carga *q*, masa $m$ que entra en un campo magnético con $\vec{v_0} \perp \vec{B} $. Dicha partícula describe un movimiento circular de radio R, velocidad angualar $w$ y periodo $T$.


a) Estudia, mediante el programa, cómo varía la frecuencia del movimiento circular de una partícula que se mueve en un campo magnético homogéneo.  Para ello usaremos el mismo programa python del selector de velocidad, quitando el campo eléctrico y extendiendo el rango en el cual actúa el campo magnético a todo el espacio y modificando las ecuaciones de movimiento. Utilizad electrones, positrones o protones con velocidades que vayan desde 10² m/s a 10⁸ m/s. Representar el movimiento de la partícula en función del tiempo, y obten la frecuencia y el radio de la trayectoria a partir de la gráfica. Deducir teóricamente estas expresiones, y hacer una gráfica en la que se comparen los resultados del programa con lo teórico. Todos estos cálculos deben presentarse en el informe así como las gráficas. 



In [1]:
# Cargamos las librerias necesarias.
import numpy as np
import math
import random
import matplotlib.pyplot as plt
#import bqplot.pyplot as plt
import matplotlib.patches as patches
import matplotlib.mlab as mlab
from scipy.integrate import odeint
from scipy.signal import find_peaks

###########################################
DNI=25431952    # introduce tu DNI sin letra
##############################################3

random.seed(DNI)
# Velocidad inicial de la particula
vx0=random.randrange(1,1000000,1)*100 # m/s

print('Velocidad inicial vx0= {:.2E} m/s'.format(vx0))

Velocidad inicial vx0= 8.69E+07 m/s


In [2]:
############## Partículas a estudiar    ###############################
# Definimos un diccionario con todas las particulas a calcular con su valor de masa y carga
# particula = {nombre : [masa (Kg), carga(C)]}
iones={'1p+': [1.67E-27, 1.6E-19],'1e-': [9.11E-31, -1.6E-19],'1e+': [9.11E-31, 1.6E-19],
       '1H+': [1.66E-27, 1.6E-19], '2H+': [3.32E-27, 1.6E-19],'3H+': [4.98E-27, 1.6E-19],'4H+': [6.688E-22, 1.6E-19],
       '3He+': [5.0052E-27, 1.6E-19],'4He+': [6.6465E-27, 1.6E-19],'4He++': [6.6465E-27, 3.2E-19],
       '12C+': [19.93E-27, 1.6E-19], '13C+': [21.59E-27, 1.6E-19], '14C+': [23.25E-27, 1.6E-19],
       '16O+': [26.56E-27, 1.6E-19], '17O+': [28.23E-27, 1.6E-19], '18O+': [29.89E-27, 1.6E-19],
       '32S+': [53.10E-27, 1.6E-19], '33S+': [54.75E-27, 1.6E-19], '34S+': [56.41E-27, 1.6E-19],
       '35Cl+': [58.10E-27, 1.6E-19], '36Cl+': [59.76E-27, 1.6E-19], '37Cl+': [61.38E-27, 1.6E-19],
       '40Ca+': [66.36E-27, 1.6E-19], '42Ca+': [69.67E-27, 1.6E-19], '43Ca+': [71.13E-27, 1.6E-19]}
ni = len(iones)

### **Cálculo analítico**##

Estudio de la frecuencia y el radio de giro de la partícula.



In [3]:

# Datos a modificar 

#m=1.  # masa del ion
#q=1.   # Carga del ion

ion='1e-'
m=iones[ion][0] # Masa del ion (Kg) 
q=iones[ion][1] # Carga del ion (C)

B=1.  # Campo magnético (T)
#vx0=7.5E7  # velocidad del io
#vx0=0.1*3E8


c=3E8  #velocidad de la luz en el vacio
g=1/np.sqrt(1-((vx0/c)*(vx0/c))) #factor gamma
mr=m*g  # si se quiere estudiar el caso relativista usar mr=m*g si no mr=m
print('Gamma= {:.3E} '.format(g))

Tnr=2*np.pi*m/q/B
Tr=2*np.pi*mr/q/B
print('Periodo no relativista= {:.3E} s'.format(Tnr))
print('Periodo relativista= {:.3E} s'.format(Tr))
print('Frecuencia no relativista= {:.3E} Hz'.format(1/Tnr))
print('Frecuencia relativista= {:.3E} Hz'.format(1/Tr))

Rnr = m*vx0/q/B
Rr = mr*vx0/q/B
print('Radio de giro no relativista= {:.3E} m'.format(Rnr))
print('Radio de giro relativista= {:.3E} m'.format(Rr))


Gamma= 1.045E+00 
Periodo no relativista= -3.577E-11 s
Periodo relativista= -3.738E-11 s
Frecuencia no relativista= -2.795E+10 Hz
Frecuencia relativista= -2.675E+10 Hz
Radio de giro no relativista= -4.949E-04 m
Radio de giro relativista= -5.171E-04 m


### **Cálculo numérico** ####
Estudio de la frecuencia y el radio de giro de la partícula.

In [4]:

# Definicion de las ecuaciones de movimiento
def circulo(z,t,par):
    x,vx,y,vy=z
    #cB=1.  
    dzdt=[vx,qm*vy*B,vy,-qm*vx*B]
    return dzdt


In [ ]:
# Este programa calcula la trayectoria circular de un ion en presencia de un
# Campo magnético perpendicular a su velocidad y luego representa
# la coordenada y del ion en funcion del tiempo


#Ecr=(np.random.rand())*40+10
#Ec=round(Ecr,2)


#c=3E8  #velocidad de la luz en el vacio
g=1/np.sqrt(1-((vx0/c)*(vx0/c))) #factor gamma
mr=m*g  # si se quiere estudiar el caso relativista usar mr=m*g si no mr=m 

# Llamada a odeint que resuleve las ecuaciones de movimiento
#tf=1.3E-8  #tiempo de simulacion. Debe ser 2 o 3 veces superior al periodo del ion para poder ver varios ciclos
tf=5*Tr

nt=10000
z0=[-0.1,vx0,0.,0.0]    
t=np.linspace(0,tf,nt)
abserr = 1.0e-8
relerr = 1.0e-6

qm=q/m
par=[qm,B]
z=odeint(circulo,z0,t,args=(par,),atol=abserr, rtol=relerr)
y = z[:,2]
vy = z[:,3]

qm=q/mr
par=[qm,B]
zr=odeint(circulo,z0,t,args=(par,),atol=abserr, rtol=relerr)
yr = zr[:,2]
vyr = zr[:,3]

###### Grafica ################################
#########  Buscamos los picos de la función ##########
max_y=np.max(y)
min_y=np.min(y)

# find_peaks(x, height=None, threshold=None, distance=None, prominence=None, width=None, wlen=None, rel_height=0.5, plateau_size=None)[source]
#peaks, _ = find_peaks(y, height=-5.0)
peaks, _ = find_peaks(y)
#print(peaks)
#print(len(peaks))

# Promedio de los periodos para todas las oscilaciones obtenidas
#T=(xf-xk)/(len(x)-1)
per=(t[peaks[-1]]-t[peaks[0]])/(len(peaks)-1)
print('T(s)  = {:.3E}'.format(per))
print('f(Hz) = {:.3E}'.format(1/per))

peaksr, _ = find_peaks(yr)
perr=(t[peaksr[-1]]-t[peaksr[0]])/(len(peaksr)-1)
print('T(s) relativista = {:.3E}'.format(perr))
print('f(Hz) relativista = {:.3E}'.format(1/perr))

fig=plt.figure()
fig.set_dpi(100)
#fig.set_size_inches(7,6.5)
# Definicion del grafico
# Modificar los limites de acuerdo a las necesidades, así como los titulos de los ejes
ax=plt.axes(xlim=(0,tf))
line1, = ax.plot(t,y,'--', linewidth=2, label='no relativista')
line1p, = ax.plot(t[peaks], y[peaks],'x')
ax.legend(loc='lower right')
ax.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))
ax.set_ylabel("y(m)")
ax.set_xlabel("t(s)")
plt.show()

fig=plt.figure()
fig.set_dpi(100)
#fig.set_size_inches(7,6.5)
# Definicion del grafico
# Modificar los limites de acuerdo a las necesidades, así como los titulos de los ejes
ax=plt.axes(xlim=(0,tf))
line1, = ax.plot(t,yr,'--', linewidth=2, label='relativista')
line1p, = ax.plot(t[peaksr], yr[peaksr],'x')
ax.legend(loc='lower right')
ax.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))
ax.set_ylabel("y(m)")
ax.set_xlabel("t(s)")
plt.show()


fig=plt.figure()
fig.set_dpi(100)
#fig.set_size_inches(7,6.5)
# Definicion del grafico
# Modificar los limites de acuerdo a las necesidades, así como los titulos de los ejes
ax=plt.axes(xlim=(0,tf))
line1, = ax.plot(t,y,'--', linewidth=2, label='no relativista')
line2, = ax.plot(t,yr,'-', linewidth=2, label='relativista')
ax.legend(loc='lower right')
ax.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))
ax.set_ylabel("y(m)")
ax.set_xlabel("t(s)")
plt.show()




**b)** Representa la frecuencia relativista y no relativista en función de la velocidad de la partícula.
Estudia cómo varía la frecuencia del movimiento circular de una partícula que se mueve en un campo magnético homogéneo en función de la velocidad. Utilizad electrones, positrones y protones con velocidades que vayan desde 10² m/s a 10⁸ m/s. Representar los resultados obtenidos en una gráfica mostrando la frecuencia en función de la velocidad, así como de la energía cinética. Deducir teóricamente estas expresiones, y hacer una gráfica en la que se comparen los resultados del programa con lo teórico. Todos estos cálculos deben presentarse en el informe así como las gráficas de la frecuencia en función de la velocidad y de la energía cinética. 

### **Cálculo analítico** ####
Definimos las funciones frequencia y obtenemos sus valores para un vector de velocidades iniciales, vxi.

In [ ]:
# Definimos las funciones para obtener las frecuencias
def freq_rel(v,B,q,m,c): 
  freq=q*B*(np.sqrt(1-(v/c)*(v/c)))/m/2/np.pi
  return freq

def freq_no(v,B,q,m):
  freq=q*B/m/2/np.pi
  return freq

# Datos a modificar 
#Ecr=(np.random.rand())*40+10
#Ec=round(Ecr,2)

# para poder ver varios ciclos
#c=3.0E8  #velocidad de la luz en el vacio
#nt= 1000
vxi=np.array([1.0E4, 1.0E6, 0.5E7, 0.5E8, 1E8, 1.5E8, 2.0E8, 2.5E8, 2.8E8])
#vxn=np.array([1.0E4, 1.0E6, 0.5E7, 0.5E8, 1E8, 1.5E8, 2.0E8, 2.5E8, 2.8E8])/c
nv=np.size(vxi)

#vx = np.linspace(0,c,nt)

# Frecuencia no relativista
fn=np.zeros(nv)
for i in range(nv):
    fn[i]=freq_no(vxi[i],B,q,m)

# Frecuencia relativista
fr=np.zeros(nv)
for i in range(nv):    
    fr[i]=freq_rel(vxi[i],B,q,m,c)


############ Grafica ##############3

fig=plt.figure()
fig.set_dpi(100)
ax=plt.axes(xlim=(0,c))
line1, =ax.plot(vxi,fn,'xr',label='Frecuencia no relativista')
line2, =ax.plot(vxi,fr,'xb',label='Frecuencia relativista')
ax.legend(loc='center left')

ax.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))

ax.set_ylabel("f(Hz)")
ax.set_xlabel("v (m/s)")

plt.show()
